In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split  
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm


In [2]:
white = pd.read_csv("white.csv")   # dataframe
whiteN = np.loadtxt('white.csv', skiprows=1, delimiter=',', dtype=np.float32)    # array
white.head()    # white.tail()

x1    x2    x3    x4     x5    x6     x7      x8    x9   x10   x11  y
0  7.0  0.27  0.36  20.7  0.045  45.0  170.0  1.0010  3.00  0.45   8.8  6
1  6.3  0.30  0.34   1.6  0.049  14.0  132.0  0.9940  3.30  0.49   9.5  6
2  8.1  0.28  0.40   6.9  0.050  30.0   97.0  0.9951  3.26  0.44  10.1  6
3  7.2  0.23  0.32   8.5  0.058  47.0  186.0  0.9956  3.19  0.40   9.9  6
4  7.2  0.23  0.32   8.5  0.058  47.0  186.0  0.9956  3.19  0.40   9.9  6

In [3]:
white.y         # white["y"]

0       6
1       6
2       6
3       6
4       6
5       6
6       6
7       6
8       6
9       6
10      5
11      5
12      5
13      7
14      5
15      7
16      6
17      8
18      6
19      5
20      8
21      7
22      8
23      5
24      6
25      6
26      6
27      6
28      6
29      7
       ..
4868    6
4869    6
4870    7
4871    6
4872    5
4873    6
4874    6
4875    6
4876    7
4877    5
4878    4
4879    6
4880    6
4881    6
4882    5
4883    6
4884    5
4885    6
4886    7
4887    7
4888    5
4889    6
4890    6
4891    6
4892    5
4893    6
4894    5
4895    6
4896    7
4897    6
Name: y, Length: 4898, dtype: int64

In [4]:
white = white[-np.isnan(white["y"])]

In [5]:
## Multiple linear regression
X = np.array(white.drop(["y"],1));  y = np.array(white["y"])
  # train data & test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [6]:
# statmodels packages
model = sm.OLS(y_train, sm.add_constant(X_train)) # sm.OLS(y,x)와 비교
results = model.fit()   # sm.OLS(y_train, sm.add_constant(X_train)).fit() 
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                     129.1
Date:                Sun, 10 Feb 2019   Prob (F-statistic):          3.70e-248
Time:                        23:33:36   Log-Likelihood:                -3878.0
No. Observations:                3428   AIC:                             7780.
Df Residuals:                    3416   BIC:                             7854.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        147.5394     21.311      6.923      0.0

In [7]:
 # sklearn package
lrm = LinearRegression(n_jobs=-1)     # 입력값형식: array-like, matrix
  # fit_intercept = True, normalize=False, n_jobs=-1 ⇨ all CPUs are used
lrm.fit(X_train, y_train)
print(lrm.intercept_ , lrm.coef_ )

147.53943573756197 [ 8.00846101e-02 -1.70374343e+00  5.69767648e-02  8.17351234e-02
 -1.64381383e-02  4.96141486e-03 -4.74149230e-04 -1.48223181e+02
  7.49786197e-01  7.33900620e-01  2.08766177e-01]


In [13]:
# 예측값
forecast = lrm.predict(X_test)
print(forecast)

[6.13867211 4.87485387 5.80282258 ... 5.75952191 6.1982174  5.35556207]


In [14]:
# MSE
mean_squared_error(y_test,forecast)

0.5679515772489915

In [10]:
## LASSO, Ridge 
lasso = Lasso(alpha=0.1); ridge = Ridge(alpha=0.1)  # alpha=0: OLS
lasso.fit(X_train, y_train)
print(lasso.intercept_ , lasso.coef_ )

3.135011905022889 [-0.         -0.          0.          0.00658765 -0.          0.00931903
 -0.0026721  -0.          0.          0.          0.26000292]


In [11]:
forecast = lasso.predict(X_test[0:10,])
print(forecast)

[5.98542298 5.39795764 5.75828732 5.94075017 6.3486235  6.35493312
 5.74667097 6.083002   5.67070478 5.79674228]


In [12]:
## training-validation procedure(반복 with train-test-split)
result = np.zeros((10,5))
a = [0.001, 0.002, 0.003, 0.004, 0.005]
for step in range(10):
   X0, X1, y0, y1 = train_test_split(X, y, test_size=0.3)
   for choice in range(len(a)):
      lasso = Lasso(alpha=a[choice], normalize=True)
         # sklearn.preprocessing.StandardScaler
      lasso.fit(X0, y0); forecast = lasso.predict(X1)
      result[step,choice] = mean_squared_error(y1,forecast)
print(result)

[[0.59444214 0.61946895 0.6535286  0.68183381 0.71239063]
 [0.58392182 0.61277532 0.65087214 0.68725191 0.72316979]
 [0.61007881 0.6298851  0.65815872 0.69311706 0.7268478 ]
 [0.59116977 0.6280598  0.67145793 0.69661399 0.72859532]
 [0.57939364 0.60740953 0.64571119 0.66831983 0.69612831]
 [0.58762979 0.61315701 0.64874907 0.67313864 0.70194655]
 [0.61765942 0.64330382 0.67878688 0.70383514 0.73258392]
 [0.63173567 0.66234333 0.70104862 0.72262381 0.75118443]
 [0.61815679 0.64750159 0.68528861 0.71603638 0.74896457]
 [0.60665083 0.6343215  0.66983755 0.68806336 0.71293532]]
